In [ ]:
from glob import glob
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import h5py
import json
from tqdm import tqdm
from copy import deepcopy
import cv2
#json template
json_template={
    "file_name":"",
    "annotations":[]#[{"bbox":[col,row,w,h],"category_id":0 or 1}]
}
#data path
def cell_intensity_classification(image, cell_list,start_x,start_y,magnification=1):
    hsi_image=cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    size=15
    for i in range(len(cell_list)):
        cell_info=cell_list[i]
        cell_image=255-hsi_image[int((cell_info['y']-start_y)/magnification)-size:int((cell_info['y']-start_y)/magnification)+size, int((cell_info['x']-start_x)/magnification)-size:int((cell_info['x']-start_x)/magnification)+size,2]
        if cell_info['cls_id']==1:
            intensity_score=np.mean(cell_image)
            
            if intensity_score>170:
                cell_list[i]['cls_id']=3
            elif intensity_score>130:
                cell_list[i]['cls_id']=2
            else:
                cell_list[i]['cls_id']=1
    return cell_list  

In [ ]:
image_count=0
train_image_list=glob('../../data/BCData/images/train/*.png')
validation_image_list=glob('../../data/BCData/images/validation/*.png')
test_image_list=glob('../../data/BCData/images/test/*.png')
os.makedirs('../../data/BCData/json_annotation',exist_ok=True)
os.makedirs('../../data/BCData/json_annotation/train/',exist_ok=True)
os.makedirs('../../data/BCData/json_annotation/validation/',exist_ok=True)
os.makedirs('../../data/BCData/json_annotation/test/',exist_ok=True)
train_positive_list=[f.replace ('images','annotations').replace('/train','/train/positive').replace('.png','.h5') for f in train_image_list]
validation_positive_list=[f.replace ('images','annotations').replace('/validation','/validation/positive').replace('.png','.h5') for f in validation_image_list]
test_positive_list=[f.replace ('images','annotations').replace('/test','/test/positive').replace('.png','.h5') for f in test_image_list]
train_negative_list=[f.replace ('images','annotations').replace('/train','/train/negative').replace('.png','.h5') for f in train_image_list]
validation_negative_list=[f.replace ('images','annotations').replace('/validation','/validation/negative').replace('.png','.h5') for f in validation_image_list]
test_negative_list=[f.replace ('images','annotations').replace('/test','/test/negative').replace('.png','.h5') for f in test_image_list]
size=15
for i in tqdm(range(len(train_image_list))):
    image_path=train_image_list[i]
    positive_path=train_positive_list[i]
    negative_path=train_negative_list[i]
    json_data=deepcopy(json_template)
    json_data['file_name']=os.path.basename(image_path)
    positive_coordinates=(h5py.File(positive_path,'r'))['coordinates'][:]
    negative_coordinates=(h5py.File(negative_path,'r'))['coordinates'][:]
    image=Image.open(image_path)
    image=np.array(image)
    hsi_image=cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    for coord in positive_coordinates:
        x,y=coord
        x1,y1=int(x)-16,int(y)-16
        w,h=32,32
        if x-16<0:
            w=32-(16-x)
            x1=0
        if y-16<0:
            h=32-(16-y)
            y1=0
        if x+16>640:
            w=32-(x+16-640)
            x1=640-w
        if y+16>640:
            h=32-(y+16-640)
            y1=640-h
        cell_image=255-hsi_image[y1:y1+h,x1:x1+w,2]
        intensity_score=np.mean(cell_image)
        if intensity_score>170:
            json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":3})
        elif intensity_score>130:
            json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":2})
        else:
            json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":1})
    for coord in negative_coordinates:
        x,y=coord
        x1,y1=int(x)-16,int(y)-16
        w,h=32,32
        if x-16<0:
            w=32-(16-x)
            x1=0
        if y-16<0:
            h=32-(16-y)
            y1=0
        if x+16>640:
            w=32-(x+16-640)
            x1=640-w
        if y+16>640:
            h=32-(y+16-640)
            y1=640-h

        json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":0})
    with open('../../data/BCData/json_annotation/train/'+os.path.basename(image_path).replace('.png','.json'),'w') as f:
        json.dump(json_data,f)
    


In [ ]:
for i in tqdm(range(len(test_image_list))):
    image_path=test_image_list[i]
    positive_path=test_positive_list[i]
    negative_path=test_negative_list[i]
    json_data=deepcopy(json_template)
    json_data['file_name']=os.path.basename(image_path)
    positive_coordinates=(h5py.File(positive_path,'r'))['coordinates'][:]
    negative_coordinates=(h5py.File(negative_path,'r'))['coordinates'][:]
    image=Image.open(image_path)
    image=np.array(image)
    hsi_image=cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    for coord in positive_coordinates:
        x,y=coord
        x1,y1=int(x)-16,int(y)-16
        w,h=32,32
        if x-16<0:
            w=32-(16-x)
            x1=0
        if y-16<0:
            h=32-(16-y)
            y1=0
        if x+16>640:
            w=32-(x+16-640)
            x1=640-w
        if y+16>640:
            h=32-(y+16-640)
            y1=640-h
        cell_image=255-hsi_image[y1:y1+h,x1:x1+w,2]
        intensity_score=np.mean(cell_image)
        if intensity_score>170:
            json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":3})
        elif intensity_score>130:
            json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":2})
        else:
            json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":1})
    for coord in negative_coordinates:
        x,y=coord
        x1,y1=int(x)-16,int(y)-16
        w,h=32,32
        if x-16<0:
            w=32-(16-x)
            x1=0
        if y-16<0:
            h=32-(16-y)
            y1=0
        if x+16>640:
            w=32-(x+16-640)
            x1=640-w
        if y+16>640:
            h=32-(y+16-640)
            y1=640-h

        json_data['annotations'].append({"bbox":[int(x1),int(y1),int(w),int(h)],"category_id":0})
    with open('../../data/BCData/json_annotation/test/'+os.path.basename(image_path).replace('.png','.json'),'w') as f:
        json.dump(json_data,f)
    

In [ ]:
plt.imshow(np.array(Image.open(train_image_list[1])))
with open('../../data/BCData/json_annotation/train/'+os.path.basename(train_image_list[1]).replace('.png','.json'),'r') as f:
    json_data=json.load(f)
for i in range(len(json_data['annotations'])):
    ann=json_data['annotations'][i]
    bbox=ann['bbox']
    category_id=ann['category_id']
    rect=plt.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],fill=False,edgecolor='r' if category_id==1 else 'b',linewidth=1)
    plt.gca().add_patch(rect)